In [1]:
import lammps
import random
from tqdm import trange
import pymc3 as pm
import arviz as az
import numpy as np
import pymatgen as pmg
import theano.tensor as tt
import glob
from copy import deepcopy
import matplotlib.pyplot as plt

from poscar_to_lammps import poscar_to_lammps
from pymatgen.io.vasp import Poscar

import sys
from contextlib import contextmanager

In [2]:
def type_shell(core_shell):
    """
    Determines which atom_type index relate to the shells on core-shell atoms
    
    Args:
        core_shell (dict): A dictionary of booleans stating if any atoms are core-shell.
    Returns:
        shell_nums (list): A list of atom_type index values for any shells present on core-shell atoms.
    """
    shell_nums = []
    x = 0
    for i in core_shell.values():
        x +=1
        if i:
            x+=1
            shell_nums.append(x)
    return shell_nums

@contextmanager
def custom_redirection(fileobj):
    """
    Redirects print to file.
    
    Args:
        fileobj (file): File name to redirect print to

    Returns:
        None.
    """
    old = sys.stdout
    sys.stdout = fileobj
    try:
        yield fileobj
    finally:
        sys.stdout = old

def structs_from_poscars(infile, outfile, core_shell, charges):
    """
    Reads in a POSCAR and creates equivalent lammps inputs.
    
    Args:
        infile (str): File path/name for POSCAR to convert.
        outfile (str): File path/name for lammps.
        core_shell (dict): A dictionary of booleans stating if any atoms should be made core-shell.
        charges (dict): A dictionary of charges for each atom type. Key = Atom label(str),
                        value = charge(float)/sub_dict(dict). If atom is core-shell a sub dictionary
                        will be the value, where sub_key = 'core' or 'shell' and sub_value = charge(float).

    Returns:
        structure (str): A lammps imput file path/name generated from the POSCAR.
    """
    structure = outfile
    with open(structure, 'w') as struct:
        with custom_redirection(struct):
            poscar = Poscar.from_file(infile)
            poscar_to_lammps(poscar, core_shell, charges )
    return structure

In [3]:
def init(structure, cs_springs=None):
    """
    Initialises the system from the structure (read in from lammps input) and non-changing parameters.
    
    Args:
        structure (str): Name of lammps input file.
        cs_springs (dict): A dictionary of the bond_coeff values. Key = atom label and
                           values = K (energy/distance^2) and r0 (equilibrium bond distance) (list).
                           Default = None.

    Returns:
        lmp (Lammps): Lammps system with structure and specified commands implemented.
    """
    lmp = lammps.Lammps(units='metal', style = 'full', args=['-log', 'none', '-screen', 'none'])
    lmp.command('read_data {}'.format(structure))

    lmp.command('group cores type 1 2 3')
    lmp.command('group shells type 4')

    if cs_springs:
        lmp.command('pair_style buck/coul/long/cs 10.0')
        lmp.command('pair_coeff * * 0 1 0')

        lmp.command('bond_style harmonic')
        for i, spring in enumerate(cs_springs):
            lmp.command('bond_coeff {} {} {}'.format(i+1,
                                                     cs_springs[spring][0],
                                                     cs_springs[spring][1]))
    else:
        lmp.command('pair_style buck/coul/long 10.0')
        lmp.command('pair_coeff * * 0 1 0')
    
    lmp.command('kspace_style ewald 1e-6')
        
    #setup for minimization
    lmp.command('min_style cg')

    return lmp

In [4]:
def update_potentials(**kwargs):
    """
    Unpdates the potentials set by pymc3 into the dictionary for the fitting process.

    Args:
        **kwargs: The parameters to be updated in the fitting process as set with pm.Model.

    Returns:
        None.
    """
    bpp.update(kwargs)
    
def set_potentials(instance):
    """
    Sets the potential for the sepecified Lammps system (changes for each iteration of the potential fit).

    Args:
        instance (Lammps): Lammps system with structure and specified commands implemented.

    Returns:
        None
    """
    for pair in pairs:
        instance.command('pair_coeff {} {} {} {} {}'.format(dex(pair[0]),
                                                            dex(pair[1]),
                                                            bpp['{}_{}_A'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_rho'.format(pair[0], pair[1])],
                                                            bpp['{}_{}_C'.format(pair[0], pair[1])]))

def dex(elin): return elements.index(elin) + 1

In [5]:
def simfunc(**kwargs):
    """
    Runs a minimization and zero step run for the instance and returns the forces.

    Args:
        **kwargs: Contain data for type of fitting and to what parameters as set with pm.Model.

    Returns:
        out (np.array): x,y,z forces on each atom.
    """
    if min(kwargs.values()) > 0:
        update_potentials(**kwargs)
        out = np.zeros([sum(core_mask),3])
        
        set_potentials(instances)

        instances.command('fix 1 cores setforce 0.0 0.0 0.0')
        instances.command('minimize 1e-25 1e-25 5000 10000')
        instances.command('unfix 1')
        instances.run(0)
        
        out = instances.system.forces[core_mask]
            
    else:
        out = np.ones([sum(core_mask),3])*999999999 # ThisAlgorithmBecomingSkynetCost

    return out

In [6]:
#Define which elements are core-shell species
core_shell = { 'Li': False , 'Ni': False, 'O': True}
shells = type_shell(core_shell)

# 2. Set charges: values either single number (non-core-shell)
# or a sub-dictionary containing { 'core': core_charge, 'shell': shell_charge }
charges = {'Li': +1.0,
           'Ni': +3.0,
           'O': {'core':  +0.960,
                 'shell': -2.960}}

infile = 'poscars/POSCAR1'
outfile = 'lammps/coords1.lmp'
structure = structs_from_poscars(infile, outfile, core_shell, charges)

elements = ['Li', 'Ni', 'Oc', 'O']
li, ni, oc, o = elements

pairs = [(li,o), (ni,o), (o,o)]

#Must be in same order given in charges dictionary
cs_springs = {o : [65.0, 0.0]} #Set to None if not using any core-shells

bpp_def = {'Li_O_A'   : 632.1018,
           'Li_O_rho' : 0.2906,
           'Li_O_C'   : 0.0,
           'Ni_O_A'   : 1582.5000,
           'Ni_O_rho' : 0.2882,
           'Ni_O_C'   : 0.000,
           'O_O_A'    : 22764.3000,
           'O_O_rho'  : 0.1490,
           'O_O_C'    : 21.7}

bpp = deepcopy(bpp_def)

In [8]:
instances = init(structure, cs_springs)
core_mask = [ atype not in shells for atype in instances.system.types]

In [14]:
expected = np.zeros([sum(core_mask),3])

In [ ]:
with pm.Model() as model:
#  pm.TruncatedNormal -- truncated so never tries a negative    pm.Uniform
    Li_O_A      = pm.Normal("Li_O_A",    mu = bpp_def['Li_O_A'],    sd =  50)
    Li_O_rho    = pm.Normal("Li_O_rho",  mu = bpp_def['Li_O_rho'],  sd =  0.05)
#     Li_O_C      = pm.Normal("Li_O_C",    mu = bpp_def['Li_O_C'],    sd =  0.01)
    Ni_O_A      = pm.Normal("Ni_O_A",    mu = bpp_def['Ni_O_A'],    sd =  100)
    Ni_O_rho    = pm.Normal("Ni_O_rho",  mu = bpp_def['Ni_O_rho'],  sd =  0.05)
#     Ni_O_C      = pm.Normal("Ni_O_C",    mu = bpp_def['Ni_O_C'],    sd =  0.05)
    O_O_A       = pm.Normal("O_O_A",     mu = bpp_def['O_O_A'],     sd =  100)
    O_O_rho     = pm.Normal("O_O_rho",   mu = bpp_def['O_O_rho'],   sd =  0.01)
    O_O_C       = pm.Normal("O_O_C",     mu = bpp_def['O_O_C'],     sd =  5)
    
    simulator = pm.Simulator('simulator', simfunc, observed=expected)
    
    trace = pm.sample(step=pm.SMC(ABC=True, epsilon=0.1), draws=1000)
#     trace = pm.sample(step=pm.SMC(ABC=True, epsilon=3000, dist_func="sum_of_squared_distance"), parallel=False, draws=1000)

In [ ]:
az.style.use('arviz-darkgrid')
az.plot_trace(trace)
plt.savefig('coreshell_LiNiO2_trace2.png',dpi=500, bbox_inches = "tight")

In [ ]:
az.plot_posterior(trace, round_to = 3, point_estimate = 'mode')
plt.savefig('coreshell_LiNiO2_mode2.png',dpi=500, bbox_inches = "tight")